In [1]:
import pandas as pd
import numpy as np

## Importando Dataset

Basta escolher o metodo referente ao dataset e tirar o comentário

In [ ]:
from LoadDataset import LoadDataset


data,target = LoadDataset.load_susy()


data = data.astype(np.float32)


ownload Susy Dataset: 922MiB [01:11, 12.9MiB/s] 

Load CSV into DataFrame


## Rodar Random Forest

In [ ]:
def run_random_forest(new_data, target, _n_estimators, _max_depth, debug=False):
      from cuml.ensemble import RandomForestClassifier
      from cuml.metrics import accuracy_score
      from cuml.model_selection import train_test_split
      import cudf
      import time
      import gc


      # Converter os dados para um DataFrame do cudf
      X_cudf = new_data
      y_cudf = cudf.Series(target)

      # Dividir os dados em conjuntos de treinamento e teste
      X_train, X_test, y_train, y_test = train_test_split(X_cudf, y_cudf, test_size=0.2, random_state=42)


      # Instanciar e treinar o modelo RandomForest
      rf = RandomForestClassifier(n_estimators=_n_estimators,
                                  max_depth = _max_depth,
                                  random_state=42)

      start_time = time.time()
      rf.fit(X_train, y_train)
      end_time = time.time()

      if debug:
        print(f'Tempo de treinamento do modelo RandomForest {_n_estimators},{_max_depth}: {end_time - start_time} segundos')

      # Fazer previsões no conjunto de teste
      y_pred = rf.predict(X_test)

      del X_train, X_test, y_train, X_cudf, y_cudf
      gc.collect()

      # Calcular a acurácia
      accuracy = accuracy_score(y_test, y_pred)
      if debug:
        print(f'Acurácia do modelo RandomForest {_n_estimators},{_max_depth}: {accuracy}')

      # Limpar a memória da GPU
      del y_test, y_pred
      gc.collect()

      return accuracy, rf

In [ ]:
import cudf

data = cudf.DataFrame(data)

# Definindo os valores para N-trees e depth
n_trees = np.arange(5, 105, 5)  # De 5 até 100, múltiplos de 5
depth = np.arange(5, 105, 5)    # De 5 até 100, múltiplos de 5

accs = []
for n in n_trees:
    for d in depth:
        acc, rf = run_random_forest(data, target, n, d, debug=True)
        accs.append((n, d, acc))
